# Prompto-Chat

In this notebook a ConversationalRetrievalChain and a custom agent are compared by results.

In [1]:
!pip install langchain openai GitPython chromadb


[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import BaseChatPromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish, HumanMessage
import re
from getpass import getpass

In [4]:
from langchain.document_loaders import GitLoader

In [5]:
loader = GitLoader(
    clone_url="https://github.com/hpi-swa-lab/godot-pronto",
    repo_path="./pronto",
    branch="master",
)

In [6]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

documents = loader.load()

In [7]:
separators = [
    # First, try to split along class definitions
    "\nclass ",
    "\nfunc ",
    "\n\tfunc ",
    # Now split by the normal type of lines
    "\n\n",
    "\n",
    " ",
    "",
]
text_splitter = RecursiveCharacterTextSplitter(separators=separators, chunk_size=512, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = Chroma.from_documents(documents=texts, embedding=embeddings)
retriever = db.as_retriever(
    search_type="mmr",  # Also test "similarity"
    search_kwargs={"k": 4},
)

In [8]:
from langchain.agents.agent_toolkits import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_codebase",
    "Searches and returns code.",
)
tools = [tool]

In [9]:
# Set up the base template
template = """Complete the objective as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

These were previous tasks you completed:



Begin!

Question: {input}
{agent_scratchpad}"""

In [10]:
# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    template: str
    # The list of tools available
    tools: List[Tool]

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts
        # Create a tools variable from the list of tools provided
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        formatted = self.template.format(**kwargs)
        return [HumanMessage(content=formatted)]

In [11]:
prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    # This omits the `agent_scratchpad`, `tools`, and `tool_names` variables because those are generated dynamically
    # This includes the `intermediate_steps` variable because that is needed
    input_variables=["input", "intermediate_steps"]
)

In [12]:
class CustomOutputParser(AgentOutputParser):

    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        # Check if agent should finish
        if "Final Answer:" in llm_output:
            return AgentFinish(
                # Return values is generally always a dictionary with a single `output` key
                # It is not recommended to try anything else at the moment :)
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        # Parse out the action and action input
        regex = r"Action\s*\d*\s*:(.*?)\nAction\s*\d*\s*Input\s*\d*\s*:[\s]*(.*)"
        match = re.search(regex, llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        # Return the action and action input
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)

In [13]:
output_parser = CustomOutputParser()

In [14]:
llm = ChatOpenAI(temperature=0, )

In [15]:
# LLM chain consisting of the LLM and a prompt
llm_chain = LLMChain(llm=llm, prompt=prompt)

In [16]:
tool_names = [tool.name for tool in tools]
agent = LLMSingleActionAgent(
    llm_chain=llm_chain,
    output_parser=output_parser,
    stop=["\nObservation:"],
    allowed_tools=tool_names
)

In [21]:
agent_executor = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, reduce_k_below_max_tokens=True)

In [22]:
agent_executor.run("Describe me how do I connect two behaviours using the pronto framework?")



> Entering new AgentExecutor chain...
Thought: I'm not familiar with the pronto framework, so I should search the codebase to find information on connecting two behaviors.

Action: search_codebase
Action Input: "pronto framework connect behaviors"

Observation:[Document(page_content='Pronto consists of a set of Godot Nodes called `Behavior` that can be added to a Godot scene. These behaviors are aspects that, when combined, result in the expression of complex behavior in the Godot scene. All behaviors have visual representation in the scene and primarily function through an event system called Connections. For example, if a timer reaches 5 seconds, a new enemy spawns.\n\n## Phase 1: Creating a Prototype', metadata={'file_name': 'README.md', 'file_path': 'README.md', 'file_type': '.md', 'source': 'README.md'}), Document(page_content='# Changelog Iteration 3 Week 2\n\n## Pronto Connection Window (Ideas for improvement)\n\n* Every time you write a character in one of the Expression Inpu

'To connect two behaviors using the Pronto framework, you can use the event system called Connections. Behaviors in Pronto are Godot Nodes that can be added to a scene. They primarily function through the Connections event system. By creating connections between behaviors, you can trigger events based on certain conditions. For example, if a timer reaches 5 seconds, a new enemy spawns.'

In [ ]:
agent_executor.run("Describe me how to setup a simple jump and run game.")

In [117]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryMemory

llm2 = ChatOpenAI()
memory = ConversationSummaryMemory(
    llm=llm, memory_key="chat_history", return_messages=True
)
qa = ConversationalRetrievalChain.from_llm(llm2, retriever=retriever, memory=memory, return_source_documents=True)

In [120]:
question = "How do I connect two Behaviours using the Pronto framework?"
result = qa(question)

ValueError: One output key expected, got dict_keys(['answer', 'source_documents'])

In [119]:
result

{'question': 'How do I connect two Behaviours using the Pronto framework?',
 'chat_history': [SystemMessage(content="The human asks how to initialize a ReAct agent. The AI provides a step-by-step guide on how to do so, including creating a new instance of the `ReActAgent` class, setting the initial state, defining behaviors, adding event handlers, and starting the agent. The AI also provides a code example to illustrate the process. The human then asks how to connect two Behaviors. The AI explains that two Behaviors can be connected using signals and slots. It provides a four-step process, which involves adding both Behaviors as child nodes to the same parent node, identifying the signal and corresponding function in each Behavior, using the `connect()` function in the parent node's script to establish the connection, and ensuring compatibility between the signal and slot arguments.")],
 'answer': 'To connect two behaviors using the Pronto framework, you can use the event system called